In [2]:
from  numba import cuda

In [1]:
import os,sys
import numpy as np
import pandas as pd
# from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
import matplotlib.pyplot as plt
sys.path.append('../LIB/')
from env import ENV
from sklearn.preprocessing import normalize
# from tqdm import tqdm
import pickle
from sklearn.preprocessing.data import QuantileTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from catboost import Pool, CatBoostRegressor
import gc


print_to_file = False 
test_run = False 

train_path = ''
test_path = ''

# train = pd.read_pickle(ENV.lightgbm_train_764.value)
train = pd.read_pickle(train_path)
print('train shape is: {}'.format(train.shape))
# test = pd.read_pickle(ENV.lightgbm_test_764.value)
test = pd.read_pickle(test_path)
print('test shape is: {}'.format(test.shape))
fe_id = 'comb_764'

FileNotFoundError: [Errno 2] No such file or directory: ''

In [ ]:
train['SK_ID_CURR'] = train['SK_ID_CURR'].astype(int)
test['SK_ID_CURR'] = test['SK_ID_CURR'].astype(int)
targets = train.TARGET.values

train_id = train['SK_ID_CURR']
test_id = test['SK_ID_CURR']

X = pd.concat([train.drop('TARGET',axis=1),test])

In [2]:
def scan_nan_portion(df):
    portions = []
    columns = []
    for col in df.columns:
        columns.append(col)
        portions.append(np.sum(df[col].isnull())/len(df))
    return pd.Series(data=portions, index=columns)


In [3]:
def fill_na_list(X,th=160):
    re = scan_nan_portion(X)
    re = re.sort_values()
    col_list = []
    na_list = []
    for col in re.index:
        if re[col] > 0 and X[col].nunique() >= th and str(X[col].dtypes)!= 'category' and str(X[col].dtypes)!= 'object':
            col_list.append(col)
            na_list.append(re[col])
    return col_list,na_list
            
def check_category_col(X,th=160):
    cate = []
    cols = list(X.columns)
    for col in cols:
        if X[col].nunique() < th or str(X[col].dtypes)== 'category' or str(X[col].dtypes)== 'object':
            cate.append(cols)
            if X[col].isnull().sum() > 0:
                print(col)
                X[col].fillna(X[col].min()-1)
    return cate


def cate_index(df,category):
    index_list = []
    for col in df.columns:
        if col in category:
            index_list.append(category.index(col))
    return index_list
    
def trainCAT(na_train_x,na_val_x,na_train_y,na_val_y):
    cat_col = cate_index(na_train_x,categorys)
    model = CatBoostRegressor(iterations=6000, 
                              depth=None, 
#                               thread_count=10,
                              learning_rate=0.01, 
                              loss_function='RMSE',
                              verbose=500,
                              task_type='GPU')
    model.fit(na_train_x,na_train_y,
              cat_features=cat_col,
              eval_set=[(na_val_x, na_val_y)],
              early_stopping_rounds=3,
              verbose_eval=500,
              metric_period=500)

#     reg.fit(na_train_x, na_train_y, eval_set=[(na_train_x,na_train_y),(na_val_x, na_val_y)],  verbose=200,early_stopping_rounds=100,eval_metric='l1' )
    return model

def saving(X):
    print('saving...')
    train_save = X.iloc[:307511].copy()
    train_save['TARGET'] = targets
    print(train_save.shape)

    test_save = X.iloc[307511:].copy()
    train_save.to_pickle(train_path)
    test_save.to_pickle(test_path)
    

In [4]:
# col_list,na_list = fill_na_list(X)
categorys = check_category_col(X)

NameError: name 'X' is not defined

In [ ]:
Processed_list = []
ignore_col = ['SK_ID_CURR']
feature_col_base = list(set(X.columns) - set(ignore_col) - set(col_list))
val = 0.1
save_count = 3
count = 0
for col,nav in zip(col_list,na_list):
    print('start processing {} \nthe na is:{}'.format(col,nav))
    X[col] = X[col].replace(365243.0,0)
    X[col] = X[col].replace(np.NINF,np.NAN)
    X[col] = X[col].replace(np.PINF,np.NAN)
    X[col] = X[col].replace(np.Inf,np.NAN)
    feature_col = list(set(feature_col_base + Processed_list))
    
    na_train = X[X[col].notnull()][feature_col].copy()
    na_target = X[X[col].notnull()][col].copy()
    seed = np.random.randint(200) + 1
    na_train_x,na_val_x,na_train_y,na_val_y = train_test_split(na_train,na_target, test_size=val,random_state=seed)
    print(na_train.shape)
    na_test = X[X[col].isnull()][feature_col].copy()
    print(na_test.shape)
    reg = trainCAT(na_train_x,na_val_x,na_train_y,na_val_y)
    X.loc[X[col].isnull(),col] = reg.predict(na_test)
    Processed_list.append(col)
    print('processed: {}/{}'.format(len(Processed_list),len(col_list)))
    count +=1
    if count % save_count == 0:
        saving(X)
    

start processing prev_DAYS_FIRST_DRAWING_minus_DAYS_FIRST_DUE_sum 
the na is:0.04883580581325175
(338857, 144)
(17398, 144)
0:	learn: 1181636.3487984	test: 1192221.0140901	best: 1192221.0140901 (0)	total: 6.37ms	remaining: 38.2s
500:	learn: 207826.9418867	test: 216924.0326676	best: 216924.0326676 (500)	total: 2.54s	remaining: 27.9s
1000:	learn: 169065.0769463	test: 177017.2484799	best: 177017.2484799 (1000)	total: 5.01s	remaining: 25s
1500:	learn: 152315.0871406	test: 160235.7132431	best: 160235.7132431 (1500)	total: 7.46s	remaining: 22.3s
2000:	learn: 142401.7961836	test: 150748.3477776	best: 150748.3477776 (2000)	total: 9.91s	remaining: 19.8s
2500:	learn: 135694.0469778	test: 144475.4785609	best: 144475.4785609 (2500)	total: 12.4s	remaining: 17.3s
3000:	learn: 130471.4499998	test: 139683.0718458	best: 139683.0718458 (3000)	total: 14.8s	remaining: 14.8s
3500:	learn: 126127.6794293	test: 135727.0450031	best: 135727.0450031 (3500)	total: 17.2s	remaining: 12.3s
4000:	learn: 122459.600159

5999:	learn: 3.5896343	test: 3.9328875	best: 3.9328875 (5999)	total: 30.2s	remaining: 0us
bestTest = 3.93288746
bestIteration = 5999
processed: 6/263
saving...
(307511, 409)
start processing all_installment_installment_paid_late_in_days_std 
the na is:0.0495291294157275
(338610, 150)
(17645, 150)
0:	learn: 27.0463942	test: 27.4716733	best: 27.4716733 (0)	total: 6.14ms	remaining: 36.9s
500:	learn: 6.3024426	test: 6.8415707	best: 6.8415707 (500)	total: 2.31s	remaining: 25.4s
1000:	learn: 4.8954512	test: 5.5234991	best: 5.5234991 (1000)	total: 4.68s	remaining: 23.4s
1500:	learn: 4.2500271	test: 4.9953936	best: 4.9953936 (1500)	total: 7.07s	remaining: 21.2s
2000:	learn: 3.8659929	test: 4.7080745	best: 4.7080745 (2000)	total: 9.48s	remaining: 18.9s
2500:	learn: 3.6034936	test: 4.5315240	best: 4.5315240 (2500)	total: 11.9s	remaining: 16.6s
3000:	learn: 3.4106225	test: 4.4163874	best: 4.4163874 (3000)	total: 14.3s	remaining: 14.3s
3500:	learn: 3.2604103	test: 4.3274049	best: 4.3274049 (3500)	

5999:	learn: 4.1635597	test: 10.2279658	best: 10.2279658 (5999)	total: 34.2s	remaining: 0us
bestTest = 10.22796583
bestIteration = 5999
processed: 12/263
saving...
(307511, 409)
start processing last_10_installment_paid_late_in_days_kurt 
the na is:0.04985473888085781
(338494, 156)
(17761, 156)
0:	learn: 1.7249234	test: 1.7296936	best: 1.7296936 (0)	total: 8.38ms	remaining: 50.3s
500:	learn: 1.0311511	test: 1.0448327	best: 1.0448327 (500)	total: 3.28s	remaining: 36s
1000:	learn: 0.9545242	test: 0.9673949	best: 0.9673949 (1000)	total: 6.18s	remaining: 30.9s
1500:	learn: 0.9144365	test: 0.9276484	best: 0.9276484 (1500)	total: 9.34s	remaining: 28s
2000:	learn: 0.8856537	test: 0.8995831	best: 0.8995831 (2000)	total: 12.3s	remaining: 24.5s
2500:	learn: 0.8647501	test: 0.8795235	best: 0.8795235 (2500)	total: 15.2s	remaining: 21.2s
3000:	learn: 0.8489062	test: 0.8647684	best: 0.8647684 (3000)	total: 18.3s	remaining: 18.3s
3500:	learn: 0.8360695	test: 0.8529582	best: 0.8529582 (3500)	total: 21

2000:	learn: 2035681261751.6669922	test: 5317516215794.3994141	best: 5268044814943.5429688 (1000)	total: 11.3s	remaining: 22.5s
2500:	learn: 1649018996891.5180664	test: 5353618405597.1074219	best: 5268044814943.5429688 (1000)	total: 14.4s	remaining: 20.1s
bestTest = 5.268044815e+12
bestIteration = 1000
processed: 19/263
start processing last_20_installment_paid_late_in_days_skew 
the na is:0.05012420878303463
(338398, 163)
(17857, 163)
0:	learn: 1.2879065	test: 1.2847620	best: 1.2847620 (0)	total: 6.5ms	remaining: 39s
500:	learn: 0.3424439	test: 0.3427232	best: 0.3427232 (500)	total: 3.4s	remaining: 37.3s
1000:	learn: 0.2748656	test: 0.2775418	best: 0.2775418 (1000)	total: 6.61s	remaining: 33s
1500:	learn: 0.2409718	test: 0.2446169	best: 0.2446169 (1500)	total: 9.56s	remaining: 28.7s
2000:	learn: 0.2193456	test: 0.2237032	best: 0.2237032 (2000)	total: 12.8s	remaining: 25.6s
2500:	learn: 0.2040018	test: 0.2089300	best: 0.2089300 (2500)	total: 15.9s	remaining: 22.2s
3000:	learn: 0.192375

In [ ]:
train = X.iloc[:307511].copy()
train['TARGET'] = target
print(train.shape)

test = X.iloc[307511:].copy()

In [ ]:
train.to_pickle(ENV.lgb_train_0827_na.value)
test.to_pickle(ENV.lgb_test_0827_na.value)